In [ ]:
import os
import gmsh
import copy
import numpy as np
import pandas as pd
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.tri import Triangulation
from scipy.constants import mu_0, epsilon_0
from scipy.sparse import lil_matrix
from scipy.special import jvp, hankel2, h2vp, jv
from fem_pre_processing import read_mesh
from fem_processing import vectorial_matrices_assembly as assembly
from fem_processing import gaussian_quadrature, master_domain
from fem_pos_processing import graph_results

## Physics Constants

In [3]:
OMEGA = 2 * np.pi * 3E8
K0 = OMEGA * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0
OPERATIONS = {'real': np.real, 'imag': np.imag, 'abs': np.abs}

# `get_edge_key_map()`

In [ ]:
def get_edge_key_map(mesh_data):
    # Construir um dicionário auxiliar para mapear arestas diretamente às suas chaves
    edge_key_map = {}
    for key, edge in mesh_data['edges'].items():
        # Usamos uma tupla ordenada para representar a aresta, garantindo unicidade independente da direção
        edge_nodes = tuple(sorted(edge['conn']))
        edge_key_map[edge_nodes] = key
    return edge_key_map

# `apply_pml_physics()`

In [ ]:
def apply_pml_physics(PML_DESIGN, mesh_data):
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']
    edges_data = mesh_data['edges']

    # Mapeamento das arestas diretamente às suas chaves    
    edge_key_map = get_edge_key_map(mesh_data)

    # Parâmetros da PML
    x0 = PML_DESIGN['x0']       # Interface do PML
    n = PML_DESIGN['n']         # Ordem do PML 
    R = PML_DESIGN['R']         # Coeficiente de reflexão do truncamento
    SIGMA_0X = -np.log(R) / WAVELENGTH

    # Adicionar as propriedades do materiais ao dicionário da célula
    for cell in cell_data.values():  
        # Atualiza a conectividade de arestas da célula
        cell['conn_edge'] = [
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][1]]))],  # e1: nó 1 -> nó 2
            edge_key_map[tuple(sorted([cell['conn_sorted'][0], cell['conn_sorted'][2]]))],  # e2: nó 1 -> nó 3
            edge_key_map[tuple(sorted([cell['conn_sorted'][1], cell['conn_sorted'][2]]))]   # e3: nó 2 -> nó 3
        ]

        # Geometric data    
        xc = np.mean([nodes_data[node]['xg'][0] for node in cell['conn']])
        yc = np.mean([nodes_data[node]['xg'][1] for node in cell['conn']])
        
        # Jacobiano da transformação isoparamétrica
        Je = np.eye(3)
        Je[:2, :2] = assembly.jacobian(mesh_data, cell)
        area = np.abs(np.linalg.det(Je)) * 0.5 

        # Atualizar as propriedades da célula
        cell['geo']['centroid'] = (xc, yc)        
        cell['geo']['jacobian'] = Je
        cell['geo']['area'] = area

        # PML x-properties
        if abs(xc) < x0:
            Sx = 1
        else:
            Sx = 1 - 1j * SIGMA_0X * (abs(xc) - x0) ** n 

        # PML y-properties
        if abs(yc) < x0:
            Sy = 1
        else:
            Sy = 1 - 1j * SIGMA_0X * (abs(yc) - x0) ** n 
        
        # Material properties
        pml_tensor = np.array([[Sy / Sx, 0, 0], [0, Sx / Sy, 0], [0, 0, 1]])
        pml_gamma = Sx * Sy

        # Material properties
        er = cell['material']['er']
        mur = cell['material']['ur']
        # sigma = cell['material']['electric_conductivity']
        # erc = er - 1j * sigma / (OMEGA * epsilon_0)
        # u_inc = np.exp(-1j * K0 * xc)

        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['material']['(Sx, Sy)'] = (Sx, Sy)
        cell['p(x)'] = (1/mur) * (1/pml_gamma)
        cell['q(x)'] = -K0**2 * er * pml_tensor
        cell['f(x)'] = np.zeros((2, 1))
        # cell['f(x)'] = K0**2 * (erc - 1) * u_inc
        # cell['f(x)'] = -(K0**2 * cell['p(x)'] + cell['q(x)']) * u_inc

        # Adicionar solução analítica no centro do elemento
        cell['ui(x)'] = np.array([[0], [1], [0]]) * np.exp(-1j * K0 * xc)

        # Inicializa os contornos do domínio
        gamma_d = {}
        for idx, node in enumerate(cell['conn']):
            bc_type = nodes_data[node]['bc']['type']

            if bc_type == 'Dirichlet':
                gamma_d[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_d'
                # nodes_data[node]['bc']['value'] = np.exp(-1j * K0 * nodes_data[node]['xg'][0])
            
        # Define o contorno do elemento, caso aplicável
        if len(gamma_d) > 1:
            cell['contour'] = {'type': 'gamma_d', 'conn_dict': gamma_d}

    # Atualiza dados físicos das arestas
    for edge in edges_data.values():
        # Nós da aresta
        nd0, nd1 = edge['conn'][0], edge['conn'][1]

        # Coordenadas dos nós inicial e final
        x0, y0, z0 = mesh_data['nodes'][nd0]['xg']
        x1, y1, z1 = mesh_data['nodes'][nd1]['xg']

        # Pontos médios da aresta
        xm, ym = (x0 + x1) / 2, (y0 + y1) / 2

        # Onda incidente transversal (vetorial) sobre a aresta
        # e_inc = np.array([[1], [0]]) * np.exp(-1j * K0 * ym)
        e_inc = np.array([[0], [1]]) * np.exp(-1j * K0 * xm)
         
        # Vetor da aresta
        ve = np.array([[x1 - x0], [y1 - y0]])

        # Vetor tangente unitário à aresta
        te = ve / np.linalg.norm(ve)
        
        # Adiciona o tamanho da aresta
        edge['len'] = np.linalg.norm(ve)
        
        # Adiciona os potenciais de Dirichlet sobre as arestas
        if nodes_data[nd0]['bc']['type'] == 'Dirichlet' and nodes_data[nd1]['bc']['type'] == 'Dirichlet':
            edge['bc'] = {
                'tag': nodes_data[nd0]['bc']['tag'],
                'type': 'Dirichlet',
                'value': (e_inc.T @ te).item(),
                'name': nodes_data[nd0]['bc']['name']}        
    
    return mesh_data

# `interpolate_solution()`

In [ ]:
def interpolate_solution(mesh_data, curl_uh, xik_master):
    # Dicionário para armazenar os potenciais vetoriais interpolados em cada célula
    uh_at_cell = {}

    # Percorre cada célula da malha
    for key, cell in mesh_data['cell'].items():    
        Je = assembly.jacobian(mesh_data, cell)    
        JinvT = np.linalg.inv(Je).T
        phi_hat = master_domain.shape_functions_n0(*xik_master)

        # Interpolação dos potenciais vetoriais em cada célula
        uh_at_cell[key] = 0
        for i, edge in enumerate(cell['conn_edge']):
            uh_at_cell[key] += curl_uh[edge].real * (JinvT @ phi_hat[i])
    
    return uh_at_cell

# `pml_local_matrices()`

In [ ]:
def pml_local_matrices(FINITE_ELEMENT, mesh_data, cell):
    # Initialize the local matrices
    Ne = len(cell['conn'])
    Se = np.zeros((Ne, Ne), dtype='complex128') # Matriz de rigidez do elemento
    Me = np.zeros((Ne, Ne), dtype='complex128') # Matriz de massa do elemento
    be = np.zeros((Ne, 1), dtype='complex128')  # Vetor de carga do elemento

    # Coordenadas do centroide do elemento
    xc, yc = cell['geo']['centroid']

    # Get the Gauss points and weights
    gauss_points, gauss_weights = gaussian_quadrature.gauss_data(FINITE_ELEMENT)
    for xik, wk in zip(gauss_points, gauss_weights):
        # Material properties
        pe = cell['p(x)']
        qe = cell['q(x)']
        fe = cell['f(x)']

        # Shape functions and derivatives
        phi, _, _, grad_phi = assembly.derivatives_at_master_domain(FINITE_ELEMENT, xik)
        
        # Jacobian and Determinant
        # Je = cell['geo']['jacobian']
        Je = assembly.jacobian(FINITE_ELEMENT, mesh_data, cell, xik) 
        Jdet, Jinv = np.abs(np.linalg.det(Je)), np.linalg.inv(Je)        
        
        # Matriz de rigidez
        Se += (Jinv @ grad_phi).T @ (pe @ Jinv @ grad_phi) * Jdet * wk

        # Matriz de massa
        Me += qe * phi.T * phi * Jdet * wk

        # Element right-hand-side vector
        be += fe * phi * Jdet * wk
    
    return Se, Me, be

# `pml_global_matrices()`

In [ ]:
def pml_global_matrices(FINITE_ELEMENT, mesh_data):
    # Inicializa a matriz global como uma matriz esparsa zero (tamanho NxN)
    Nnodes = len(mesh_data['nodes'])
    Sg = lil_matrix((Nnodes, Nnodes), dtype='complex128')
    Mg = lil_matrix((Nnodes, Nnodes), dtype='complex128')
    bg = lil_matrix((Nnodes, 1), dtype='complex128')

    # Início do processo de montagem
    for cell in mesh_data['cell'].values(): 
        Se, Me, be = pml_local_matrices(FINITE_ELEMENT, mesh_data, cell)

        # loop sobre os nós locais de cada elemento
        for i, ig in enumerate(cell['conn']):
            ig = int(ig)-1
            for j, jg in enumerate(cell['conn']):
                jg = int(jg)-1
                Sg[ig, jg] += Se[i, j]
                Mg[ig, jg] += Me[i, j]
                
            # preenche o vetor global b
            bg[ig, 0] += be[i, 0]

    return Sg, Mg, bg

# `create_domain()`

In [ ]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, GEOMETRY, auto_save=True, view_mesh=False):
    # Parâmetros do elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    mesh_data = {}

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = GEOMETRY['h']     # Tamanho do elemento finito
    L = GEOMETRY['L']     # Espessura da PML
    ra = GEOMETRY['ra']   # Raio do espalhador
    x0 = GEOMETRY['x0']   # Fronteira interna da PML
    xa = x0
    xb = x0 + L

    # Criar regiões absorvedoras, omega_PML
    region_i = factory.addRectangle(-xa, -xb, 0, 2*x0, L)
    region_a = factory.addRectangle(-xb, -xb, 0, L, L)
    region_b = factory.addRectangle(xa, -xb, 0, L, L)
    region_ii = factory.addRectangle(xa, -xa, 0, L, 2*x0)
    region_c = factory.addRectangle(xa, xa, 0, L, L)
    region_iii = factory.addRectangle(-xa, xa, 0, 2*x0, L)
    region_d = factory.addRectangle(-xb, xa, 0, L, L)
    region_iv = factory.addRectangle(-xb, -xa, 0, L, 2*x0)

    # Criar região do espaço livre, omega_fs
    region_fs = factory.addRectangle(-xa, -xa, 0, 2*x0, 2*x0)

    # Fragmentar todas as regiões para garantir interfaces conformais
    objectDimTags = [
        (2, region_fs),
        (2, region_i), (2, region_ii), (2, region_iii), (2, region_iv), 
        (2, region_a), (2, region_b), (2, region_c), (2, region_d)
    ]
    
    factory.fragment(objectDimTags, objectDimTags)
    
    # Criar região do espalhador, omega_s
    disk = factory.addDisk(0, 0, 0, ra, ra)

    # Subtrair omega_s de omega_fs
    outDimTags_omega_s, _ = factory.cut([(2, region_fs)], [(2, disk)], removeTool=True)
        
    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter o contorno (curva, dim=1) de gamma_s
    DimTags_free_space = gmsh.model.getBoundary(outDimTags_omega_s, oriented=True, recursive=False)

    # TAGs de gamma_s
    TagList_scatterer = [-tag[1] for tag in DimTags_free_space if tag[1] < 0]

    # TAGs de gamma_fs
    tagList_fs = [tag[1] for tag in DimTags_free_space if tag[1] > 0]

    # Adicionar grupos físicos para curvas (Dim=1)
    for i, CurveTagList in enumerate([TagList_scatterer]):
        gmsh.model.addPhysicalGroup(1, CurveTagList, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)	    
    TagList_surfaces = [region_fs, region_a, region_b, region_c, region_d, region_i, region_ii, region_iii, region_iv]

    for i, SurfaceList in enumerate(TagList_surfaces):
        gmsh.model.addPhysicalGroup(2, [SurfaceList], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(ElementOrder)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_pml_domain_{ElementType}{ElementOrder}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY, problem_dim=2)
    mesh_data['edges'] = read_mesh.get_edge_data()
    
    gmsh.finalize()
    return mesh_data

# Computation Domain, $\Omega_c$

## `ez_at_node()`

In [9]:
def ez_at_node(node, ra, series_terms=40):
    # Converter coordenadas cartesianas para coordenadas polares
    ez = 0    
    x, y = node['xg']
    rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)
    for n in range(0, series_terms):
        e_n = 2 if n != 0 else 1
        Jn_ka = jv(n, K0 * ra)
        Hn2_ka = hankel2(n, K0 * ra)
        Hn2_kp = hankel2(n, K0 * rho)
        ez += (-1j) ** n * e_n * (Jn_ka / Hn2_ka) * Hn2_kp * np.cos(n * phi)
    return ez

## `ezh_at_contour()`

In [11]:
def ez_at_contour(mesh_data, contour='BGT'):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        if node['bc']['type'] == contour:
            # Coordenadas cartesianas do nó
            x, y = node['xg']
            
            # Converter coordenadas cartesianas para coordenadas polares
            rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)

            # Converter phi para o intervalo [0, 2*pi]
            if phi < 0:
                phi = 2 * np.pi + phi

            # Analytical solution
            ez[key] = {
                'phi': phi,
                'ez': ez_at_node(node)
            }
    return ez

## `calculate_error()`

In [12]:
def calculate_error(u_ex, u_h):
    """
    Calcula o erro entre a solução exata e a solução aproximada.

    Parâmetros:
    - u_exact: Array com os valores da solução exata u_ex.
    - u_approx: Array com os valores da solução aproximada u_h.

    Retorna:
    - erro: Valor do erro calculado.
    """
    # Verificar se os arrays têm o mesmo tamanho
    if len(u_ex) != len(u_h):
        raise ValueError("Os arrays u_exact e u_approx devem ter o mesmo tamanho.")

    # Calcular a soma dos quadrados das diferenças
    N = len(u_ex)
    return np.abs(np.sqrt(np.sum((u_ex - u_h)**2) / N))

## `plot_coordinates()`

In [13]:
def plot_coordinates(mesh_data):
    pec_coords = []         # Coordenadas de nós do espalhador
    inn_pml_coords = []     # Coordenadas de nós da fronteira interna da PML

    for node in mesh_data['nodes'].values():
        x, y = node['xg']
        if node['bc']['tag'] == 101:
            pec_coords.append((x, y))
        elif node['bc']['tag'] == 102:
            inn_pml_coords.append((x, y))        

    # Conversão para listas separadas de x e y
    pec_x, pec_y = zip(*pec_coords) if pec_coords else ([], [])
    inn_pml_x, inn_pml_y = zip(*inn_pml_coords) if inn_pml_coords else ([], [])

    # Visualização
    plt.figure(figsize=(8, 8))
    plt.scatter(pec_x, pec_y, color='blue', label='Scatterer', marker='o', s=10)
    plt.scatter(inn_pml_x, inn_pml_y, color='red', label='Inner PML', marker='x', s=10)
    plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Visualização de Nós por Tipo de Condição de Contorno")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# Physical Domain, $\Omega_{fs}$

## `get_physical_mesh_data()`

In [ ]:
def get_physical_mesh_data(mesh_data):
    mesh_data_copy = copy.deepcopy(mesh_data)

    # Estruturas de dados para a malha física
    physical_mesh_data = {'cell': {}, 'nodes': {}, 'edges': {}}

    # Filtrar células do domínio físico
    for key, cell in mesh_data_copy['cell'].items():
        if cell['material']['tag'] == 201:
            physical_mesh_data['cell'][key] = cell
            
            # Nós da célula
            for node in cell['conn']:
                physical_mesh_data['nodes'][node] = mesh_data_copy['nodes'][node]

            # Arestas da célula
            for edge in cell['conn_edge']:
                physical_mesh_data['edges'][edge] = mesh_data_copy['edges'][edge]


    # Ordenar as chaves do dicionário 'nodes'
    physical_mesh_data['nodes'] = dict(sorted(physical_mesh_data['nodes'].items()))

    # Recriar conectividade com índices ajustados
    node_index_map = {node: idx+1
                    for idx, node in enumerate(physical_mesh_data['nodes'].keys())}
    
    # Atualizar a conectividade das células
    for cell in physical_mesh_data['cell'].values():
        cell['conn_physical'] = [node_index_map[node] for node in cell['conn']]
    
    # Ordenar as chaves do dicionário 'edges'
    physical_mesh_data['edges'] = dict(sorted(physical_mesh_data['edges'].items()))

    # Recriar conectividade com índices ajustados
    edge_index_map = {edge: idx+1
                    for idx, edge in enumerate(physical_mesh_data['edges'].keys())}
    
    # Atualizar a conectividade das células
    for cell in physical_mesh_data['cell'].values():
        cell['conn_edge_physical'] = [edge_index_map[edge] for edge in cell['conn_edge']]

    return physical_mesh_data

## `physical_structured_data()`

In [ ]:
def physical_structured_data(physical_mesh_data):
    # Extrair coordenadas dos nós físicos
    xg_phy = [node['xg'][0] for node in physical_mesh_data['nodes'].values()]
    yg_phy = [node['xg'][1] for node in physical_mesh_data['nodes'].values()]

    # Recriar conectividade com índices ajustados
    conn_py_phy = [[node - 1 for node in cell['conn_physical'][:3]]
                    for cell in physical_mesh_data['cell'].values()]
    
    return xg_phy, yg_phy, conn_py_phy

## `plot_physical_mesh()`

In [ ]:
def plot_physical_mesh(FINITE_ELEMENT, INFO_GRAPH, mesh_data, physical_mesh_data, name='Entire'):
    # Dados do gráfico
    show_cell = INFO_GRAPH['cell']
    show_nodes = INFO_GRAPH['nodes']
    show_edges = INFO_GRAPH['edges']
    show_edges_numb = INFO_GRAPH['edges_numb']

    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT

    # Estruturando os dados da malha
    physical_nodes = physical_mesh_data['nodes']
    physical_cells = physical_mesh_data['cell']

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)

    plt.figure(figsize=(8, 6))
    if ElementType == "Triangle":
        # Plotando a malha de elementos finitos
        plt.triplot(xg_phy, yg_phy, conn_py_phy, color='gray')  

        # Plotando as arestas dos elementos
        if show_edges or show_edges_numb:
            for key, edge in physical_mesh_data['edges'].items():
                # Coordenadas dos nós inicial e final
                x0 = physical_nodes[edge['conn'][0]]['xg'][0]
                y0 = physical_nodes[edge['conn'][0]]['xg'][1]
                x1 = physical_nodes[edge['conn'][1]]['xg'][0]
                y1 = physical_nodes[edge['conn'][1]]['xg'][1]
                
                # Ponto médio da aresta
                x_mid, y_mid = (x0 + x1) / 2, (y0 + y1) / 2
                
                # Vetor da seta (a partir do ponto médio)
                dx, dy = (x1 - x0) * 0.2, (y1 - y0) * 0.2  
                    
                # Adicionando uma seta no meio da aresta
                if show_edges:
                    plt.arrow(x_mid, y_mid, dx, dy, head_width=0.015, head_length=0.05,
                        fc='blue', ec='blue', length_includes_head=True)

                # Adicionando os números das arestas
                if show_edges_numb:
                    plt.scatter(x_mid, y_mid, marker='s', color='white', edgecolor='black', s=120, zorder=1)                
                    plt.text(x_mid, y_mid, key, color='blue', fontsize=6, ha='center', va='center')

        # Adicionando nós
        if show_nodes:            
            for key, node in physical_nodes.items():
                x, y = node['xg'][0], node['xg'][1]
                plt.scatter(x, y, color='white', edgecolor='black', s=180)
                plt.text(x, y, str(key), color='red', fontsize=8, ha='center', va='center')
        else:
            plt.scatter(xg_phy, yg_phy, color='black', s=1, zorder=3)

        # Adicionando elementos
        if show_cell:
            for key, cell in physical_cells.items():
                xc = np.mean([physical_nodes[node]['xg'][0] for node in cell['conn']])
                yc = np.mean([physical_nodes[node]['xg'][1] for node in cell['conn']])
                plt.text(xc, yc, str(key), fontweight='bold', color='black', fontsize=9, ha='center', va='center')

    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pre_processing/pictures/meshed_physical_domain_{ElementType}{ElementOrder}_{name}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

## `solution_at_physical_domain()`

In [ ]:
def solution_at_physical_domain(FINITE_ELEMENT, GEOMETRY, physical_mesh_data, domain='Entire', type='abs'):
    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT
    ra = GEOMETRY['ra']
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)

    # Cálculo da solução analítica
    ez = {key: ez_at_node(node, ra)
           for key, node in physical_mesh_data['nodes'].items()}

    # Conversão do dicionário de soluções para lista
    if type in OPERATIONS:
        ez_list = OPERATIONS[type](list(ez.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Verificação do tipo e ordem do elemento
    if ElementType == "Triangle":
        triangulation = Triangulation(xg_phy, yg_phy, conn_py_phy)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, ez_list, cmap='viridis')
    plt.colorbar(label=r'$|E_z^s(x, y)|$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')

    # Salvando o gráfico
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution_at_physical_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    return ez

## `fem_solution_at_physical_domain()`

In [ ]:
def fem_solution_at_physical_domain(FINITE_ELEMENT, mesh_data, physical_mesh_data, ezh, domain='Entire', type='abs'):
    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)
    
    # Filtrar os nós do domínio físico
    ezh_phy = {key: ezh[key] 
               for key in mesh_data['nodes'] if key in physical_mesh_data['nodes']}

    # Conversão do dicionário de soluções para lista
    if type in OPERATIONS:
        ezh_list = OPERATIONS[type](list(ezh_phy.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")
    
    # Verificação do tipo e ordem do elemento
    if ElementType == "Triangle":
        triangulation = Triangulation(xg_phy, yg_phy, conn_py_phy)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, ezh_list, cmap='viridis')
    plt.colorbar(label=r'$|{Ez}^s_h(x, y)|$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')

    # Salvando o gráfico
    filepath = graph_results.get_dir(f"pos_processing/pictures/fem_solution_at_physical_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    return ezh_phy

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.